In [1]:
import numpy as np
import torch
from collections import namedtuple

torch.__version__, np.__version__

('1.10.0', '1.21.4')

## Constants

In [2]:
c_light = 2.99792458e8 #speed of light in m/s
mc2 = 0.510998950e6 #electron mass in eV

# Drift element tuple

In [3]:
Drift = namedtuple('Drift', 'L p0c')
L = 1.000
p0c = 10.0e6 #Reference particle momentum in eV
d1 = Drift(L, p0c)
d1

Drift(L=1.0, p0c=10000000.0)

# Particle tuple

In [4]:
Particle = namedtuple('Particle', 'x px y py z pz')
#Canonical phase space coordinates as defined in Bmad manual section 15.4.2.

In [5]:
np.random.seed(0)
pvec = np.random.rand(10_000, 6) #test particles

## NumPy array test Particles

In [6]:
p_np = Particle(*pvec.T)
p_np

Particle(x=array([0.5488135 , 0.43758721, 0.56804456, ..., 0.16739177, 0.8283866 ,
       0.42741876]), px=array([0.71518937, 0.891773  , 0.92559664, ..., 0.3514515 , 0.35342665,
       0.88324715]), y=array([0.60276338, 0.96366276, 0.07103606, ..., 0.27008432, 0.92013388,
       0.38008123]), py=array([0.54488318, 0.38344152, 0.0871293 , ..., 0.08702639, 0.81374799,
       0.16900413]), z=array([0.4236548 , 0.79172504, 0.0202184 , ..., 0.30485812, 0.68327109,
       0.26393067]), pz=array([0.64589411, 0.52889492, 0.83261985, ..., 0.13144231, 0.86909734,
       0.30116676]))

## PyTorch tensor test Particles

In [7]:
tp0c = torch.tensor(p0c, dtype=torch.float64)
tmc2 = torch.tensor(mc2, dtype=torch.float64) #necessary? 
tpvec = torch.tensor(pvec, requires_grad=True, dtype=torch.float64)
p_torch = Particle(*tpvec.T)
p_torch

Particle(x=tensor([0.5488, 0.4376, 0.5680,  ..., 0.1674, 0.8284, 0.4274],
       dtype=torch.float64, grad_fn=<UnbindBackward0>), px=tensor([0.7152, 0.8918, 0.9256,  ..., 0.3515, 0.3534, 0.8832],
       dtype=torch.float64, grad_fn=<UnbindBackward0>), y=tensor([0.6028, 0.9637, 0.0710,  ..., 0.2701, 0.9201, 0.3801],
       dtype=torch.float64, grad_fn=<UnbindBackward0>), py=tensor([0.5449, 0.3834, 0.0871,  ..., 0.0870, 0.8137, 0.1690],
       dtype=torch.float64, grad_fn=<UnbindBackward0>), z=tensor([0.4237, 0.7917, 0.0202,  ..., 0.3049, 0.6833, 0.2639],
       dtype=torch.float64, grad_fn=<UnbindBackward0>), pz=tensor([0.6459, 0.5289, 0.8326,  ..., 0.1314, 0.8691, 0.3012],
       dtype=torch.float64, grad_fn=<UnbindBackward0>))

# track_a_drift

In [8]:
def make_track_a_drift(lib):
    """
    Makes track_a_drift given the library lib
    """
    
    sqrt = lib.sqrt
    
    def track_a_drift(p_in, drift):
        """
        Tracks the incoming Particle p_in though drift element
        and returns the outgoing particle. See eqs 24.58 in bmad manual
        """
        L = drift.L
        p0c = drift.p0c
        
        x = p_in.x
        px = p_in.px
        y = p_in.y
        py = p_in.py
        z = p_in.z
        pz = p_in.pz
        
        P = 1 + pz #Particle's total momentum over p0
        Px = px / P #Particle's 'x' momentum over p0
        Py = py / P #Particle's 'y' momentum over p0
        Pxy2 = Px**2 + Py**2 #Particle's transverse mometum^2 over p0^2
        Pl = sqrt(1-Pxy2)  #Particle's longitudinal momentum over p0
        
        x = x + L * Px / Pl
        y = y + L * Py / Pl
        
        beta = sqrt( 1 - 1 /( 1 + (P*p0c/mc2)**2 ) )
        beta_ref = sqrt( 1 - 1 /( 1 + (p0c/mc2)**2 ) )
        z = z + L * ( beta/beta_ref - 1.0/Pl ) #should fix by taking the effect of include_ref_motion (see eq 24.58)
        #s = s + L 
        #t = t + L / ( beta * Pl * c_light )
        
        return Particle(x, px, y, py, z, pz)
    
    return track_a_drift

# Specialized functions
track_a_drift_torch = make_track_a_drift(lib=torch)

## PyTorch jacobian example

In [9]:
from torch.autograd.functional import jacobian

In [10]:
td1 = Drift(torch.tensor(L, dtype=torch.float64),torch.tensor(p0c, dtype=torch.float64))
td1

Drift(L=tensor(1., dtype=torch.float64), p0c=tensor(10000000., dtype=torch.float64))

In [11]:
pvec1 = (0.5, 0.02, 0.5, -0.15, 0.1, 0.1)
tvec1 = torch.tensor(pvec1, requires_grad=True, dtype=torch.float64)
f2 = lambda x: track_a_drift_torch(Particle(*x), td1)
J = jacobian(f2, tvec1)
J

(tensor([ 1.0000,  0.9181,  0.0000, -0.0023,  0.0000, -0.0170],
        dtype=torch.float64),
 tensor([0., 1., 0., 0., 0., 0.], dtype=torch.float64),
 tensor([ 0.0000, -0.0023,  1.0000,  0.9352,  0.0000,  0.1276],
        dtype=torch.float64),
 tensor([0., 0., 0., 1., 0., 0.], dtype=torch.float64),
 tensor([ 0.0000, -0.0170,  0.0000,  0.1276,  1.0000,  0.0197],
        dtype=torch.float64),
 tensor([0., 0., 0., 0., 0., 1.], dtype=torch.float64))

In [12]:
track_a_drift_torch(Particle(*tvec1), td1)

Particle(x=tensor(0.5184, dtype=torch.float64, grad_fn=<AddBackward0>), px=tensor(0.0200, dtype=torch.float64, grad_fn=<UnbindBackward0>), y=tensor(0.3623, dtype=torch.float64, grad_fn=<AddBackward0>), py=tensor(-0.1500, dtype=torch.float64, grad_fn=<UnbindBackward0>), z=tensor(0.0906, dtype=torch.float64, grad_fn=<AddBackward0>), pz=tensor(0.1000, dtype=torch.float64, grad_fn=<UnbindBackward0>))

# Quadrupole

In [13]:
Quadrupole = namedtuple('Quadrupole', 'k b ')
L = 1.000
p0c = 10.0e6 #Reference particle momentum in eV
d1 = Drift(L, p0c)
d1

Drift(L=1.0, p0c=10000000.0)

In [14]:
def make_track_a_quadrupole(lib):
    """
    Makes track_a_drift given the library lib
    """
    
    sqrt = lib.sqrt
    sin = lib.sin
    cos = lib.cos
    sinh = lib.sinh
    cosh = lib.cosh
    
    def track_a_quadrupole(p_in, quad):
        """
        Tracks the incoming Particle p_in though quadrupole element
        and returns the outgoing particle. 
        """
        L = quad.L
        p0c = drift.p0c
        
        x = p_in.x
        px = p_in.px
        y = p_in.y
        py = p_in.py
        z = p_in.z
        pz = p_in.pz
        
        P = 1 + pz #Particle's total momentum over p0
        Px = px / P #Particle's 'x' momentum over p0
        Py = py / P #Particle's 'y' momentum over p0
        Pxy2 = Px**2 + Py**2 #Particle's transverse mometum^2 over p0^2
        Pl = sqrt(1-Pxy2)  #Particle's longitudinal momentum over p0
        
        x = x + L * Px / Pl
        y = y + L * Py / Pl
        
        beta = sqrt( 1 - 1 /( 1 + (P*p0c/mc2)**2 ) )
        beta_ref = sqrt( 1 - 1 /( 1 + (p0c/mc2)**2 ) )
        z = z + L * ( beta/beta_ref - 1.0/Pl ) #should fix by taking the effect of include_ref_motion (see eq 24.58)
        #s = s + L 
        #t = t + L / ( beta * Pl * c_light )
        
        return Particle(x, px, y, py, z, pz)
    
    return track_a_quadrupole

# Specialized functions
track_a_quadrupole_torch = make_track_a_quadrupole(lib=torch)